<a href="https://colab.research.google.com/github/Yoon-jeongwoo/repository/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import re

In [ ]:

# CSV 파일 불러오기
file1_path = '/content/drive/MyDrive/twcs/sample.csv'
file2_path = '/content/drive/MyDrive/twcs/twcs.csv'
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# response_tweet_id와 in_response_to_tweet_id 열의 아이디 추출
response_tweet_ids_df1 = df1['response_tweet_id'].unique()
in_response_to_tweet_ids_df1 = df1['in_response_to_tweet_id'].unique()

response_tweet_ids_df2 = df2['response_tweet_id'].unique()
in_response_to_tweet_ids_df2 = df2['in_response_to_tweet_id'].unique()

# NaN 값을 0으로 대체
response_tweet_ids_df1 = pd.Series(response_tweet_ids_df1).fillna(0)
in_response_to_tweet_ids_df1 = pd.Series(in_response_to_tweet_ids_df1).fillna(0)

response_tweet_ids_df2 = pd.Series(response_tweet_ids_df2).fillna(0)
in_response_to_tweet_ids_df2 = pd.Series(in_response_to_tweet_ids_df2).fillna(0)

# 아이디별 개수 카운트를 위한 딕셔너리 초기화
response_tweet_id_counts_df1 = {}
in_response_to_tweet_id_counts_df1 = {}

response_tweet_id_counts_df2 = {}
in_response_to_tweet_id_counts_df2 = {}

# response_tweet_id 열의 아이디별 개수 카운트
for tweet_id in response_tweet_ids_df1:
    count = df1['response_tweet_id'].value_counts()[tweet_id]
    response_tweet_id_counts_df1[tweet_id] = count

for tweet_id in response_tweet_ids_df2:
    count = df2['response_tweet_id'].value_counts()[tweet_id]
    response_tweet_id_counts_df2[tweet_id] = count

# in_response_to_tweet_id 열의 아이디별 개수 카운트
for tweet_id in in_response_to_tweet_ids_df1:
    count = df1['in_response_to_tweet_id'].value_counts()[tweet_id]
    in_response_to_tweet_id_counts_df1[tweet_id] = count

for tweet_id in in_response_to_tweet_ids_df2:
    count = df2['in_response_to_tweet_id'].value_counts()[tweet_id]
    in_response_to_tweet_id_counts_df2[tweet_id] = count

# 결과 출력
print("File 1 - response_tweet_id 카운트:")
print(response_tweet_id_counts_df1)

print("\nFile 1 - in_response_to_tweet_id 카운트:")
print(in_response_to_tweet_id_counts_df1)

print("\nFile 2 - response_tweet_id 카운트:")
print(response_tweet_id_counts_df2)

print("\nFile 2 - in_response_to_tweet_id 카운트:")
print(in_response_to_tweet_id_counts_df2)




In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 데이터 불러오기
df1 = pd.read_csv('/content/drive/MyDrive/twcs/sample.csv')
df2 = pd.read_csv('/content/drive/MyDrive/twcs/twcs.csv')

# 필요한 열 선택
df1 = df1[['text']]
df2 = df2[['text']]

# 두 개의 데이터프레임을 합치기
df = pd.concat([df1, df2], ignore_index=True)

# 랜덤 샘플링으로 데이터 양 조정
sampled_df = df.sample(n=10000, random_state=42)

# 감정분석 레이블 설정
sampled_df['sentiment'] = '중립'  # 중립으로 일단 초기화

# 감정분석에 사용할 문장 목록
positive_keywords = ['행복', '기쁨', '즐거움', '사랑', '감사']
negative_keywords = ['슬픔', '우울', '지루함', '짜증', '분노']

# 키워드에 따라 감정분석 레이블 설정
for idx, row in sampled_df.iterrows():
    text = row['text']
    for keyword in positive_keywords:
        if keyword in text:
            sampled_df.at[idx, 'sentiment'] = '긍정'
            break
    for keyword in negative_keywords:
        if keyword in text:
            sampled_df.at[idx, 'sentiment'] = '부정'
            break

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(sampled_df['text'], sampled_df['sentiment'], test_size=0.2, random_state=42)

# 벡터화
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# 분류 모델 학습
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# 테스트 데이터로 예측
y_pred = model.predict(X_test_vectorized)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"정확도: {accuracy:.4f}")

정확도: 1.0000


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# CSV 파일 불러오기 (전체 데이터 사용)
df1 = pd.read_csv('/content/drive/MyDrive/twcs/sample.csv')
df2 = pd.read_csv('/content/drive/MyDrive/twcs/twcs.csv')

# 샘플링 개수 설정 (전체 데이터를 사용하려면 n=len(df1) 또는 n=len(df2)로 설정)
num_samples = 10000

# 테스트를 위해 데이터 샘플링 (중복을 허용하여 샘플링)
df1_sampled = df1.sample(n=num_samples, replace=True)
df2_sampled = df2.sample(n=num_samples, replace=True)

# 텍스트 데이터 추출
text_data1 = df1_sampled['text'].tolist()
text_data2 = df2_sampled['text'].tolist()

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix1 = vectorizer.fit_transform(text_data1)
tfidf_matrix2 = vectorizer.transform(text_data2)

# 코사인 유사도 계산
similarity_matrix = cosine_similarity(tfidf_matrix1, tfidf_matrix2)

# 유사도 행렬 출력
print(similarity_matrix)


[[0.02663324 0.         0.         ... 0.         0.         0.01841711]
 [0.10334961 0.         0.         ... 0.09784552 0.07391343 0.        ]
 [0.10156838 0.         0.         ... 0.09615915 0.07263953 0.        ]
 ...
 [0.02198646 0.         0.         ... 0.         0.0577396  0.01520382]
 [0.07758877 0.         0.         ... 0.01850924 0.01601298 0.10442264]
 [0.02357349 0.02703565 0.         ... 0.01363411 0.09152957 0.01630127]]


In [4]:
# 대화 데이터 샘플
conversation_data = df2_sampled[['text', 'response_tweet_id']]

# 챗봇 함수
def chatbot(input_text, conversation_data):
    # 입력 문장의 TF-IDF 벡터화
    input_vector = vectorizer.transform([input_text])

    # 유사도 계산
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix2)

    # 가장 유사한 대화 선택
    max_similarity_idx = similarity_scores.argmax()
    response = conversation_data.iloc[max_similarity_idx]['response_tweet_id']

    return response

# 챗봇 실행
print("챗봇: 안녕하세요! 무엇을 도와드릴까요?")
while True:
    user_input = input("사용자: ")
    if user_input == "안녕":
        print("챗봇: 안녕히 가세요!")
        break
    response = chatbot(user_input, conversation_data)
    print(f"챗봇: {response}")


챗봇: 안녕하세요! 무엇을 도와드릴까요?
사용자: ㅇㄹ
챗봇: 981890
사용자: 안녕
챗봇: 안녕히 가세요!


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# 불용어 리스트 정의
stop_words = set(stopwords.words('english'))

# 데이터 혼합
conversation_data = pd.concat([df1[['text', 'in_response_to_tweet_id']], df2[['text', 'in_response_to_tweet_id']]])

# 데이터 전처리 함수 (불용어 처리 추가)
def preprocess_text(text):
    tokens = text.split()
    preprocessed_tokens = [token for token in tokens if token not in stop_words]
    preprocessed_text = ' '.join(preprocessed_tokens)
    return preprocessed_text

# 텍스트 전처리
conversation_data['text_preprocessed'] = conversation_data['text'].apply(preprocess_text)

# 샘플링
sample_size = 5000  # 샘플링할 데이터 개수
sampled_data = conversation_data.sample(sample_size, random_state=42)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sampled_data['text_preprocessed'])

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(sampled_data['in_response_to_tweet_id'])

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

# 모델 구성
classifier = SVC(kernel='linear')
model = Pipeline([('classifier', classifier)])

# 모델 학습
model.fit(X_train, y_train)

# 모델 평가
accuracy = model.score(X_test, y_test)
print(f"모델 정확도: {accuracy}")

# 챗봇 실행
print("챗봇: 안녕하세요! 무엇을 도와드릴까요?")
while True:
    user_input = input("사용자: ")
    if user_input == "안녕":
        print("챗봇: 안녕히 가세요!")
        break
    # 사용자 입력 전처리
    preprocessed_input = preprocess_text(user_input)
    # TF-IDF 벡터화
    tfidf_input = vectorizer.transform([preprocessed_input])
    # 의도 분류
    intent = model.predict(tfidf_input)
    # 레이블을 텍스트로 변환
    response_intent = label_encoder.inverse_transform(intent)[0]
    # 분류된 의도에 따라 적절한 응답 출력
    response_data = conversation_data[conversation_data['in_response_to_tweet_id'] == response_intent]
    if not response_data.empty:
        response = response_data['text'].iloc[0]
        print(f"챗봇: {response}")
    else:
        print("챗봇: 죄송합니다. 제가 도와드릴 수 있는 정보가 없네요.")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


모델 정확도: 0.3
챗봇: 안녕하세요! 무엇을 도와드릴까요?
사용자: If anything comes up, just let us know.
챗봇: 죄송합니다. 제가 도와드릴 수 있는 정보가 없네요.
사용자: 안녕
챗봇: 안녕히 가세요!
